# **Dynamical chemostat**
by: Edwin Saavedra C.

In [1]:
#%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import ipywidgets as wd
import sympy as sp
import pandas as pd
from IPython.display import display

In [2]:
## MATPLOTLIB rcParams
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.labelweight'] = 'medium'
plt.rcParams['axes.labelpad'] = 5.0
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['axes.edgecolor'] = 'gray'

In [3]:
def buildEquations(pi):
    x,s = sp.symbols('x s')
    p0,p1,p2,p3 = sp.symbols('p_0 p_1 p_2 p_3')

    dxdtEq = sp.Eq(x*s/(p0+s) - p1*x - p2*x,0)
    dsdtEq = sp.Eq(p2*(1-s) - p3*x*s/(p0+s),0)
#    sns = sp.solve([dxdtEq,dcdtEq],(x,s))
    return dxdtEq,dsdtEq

In [4]:
def stream(x,s,p):
    dxdt =  x*s/(p[0]+s) - p[1]*x - p[2]*x
    dsdt =  p[2]*(1-s) - p[3]*x*s/(p[0]+s)
    return dxdt,dsdt

def steadyState(p):
    ssX = p[2]*(p[0]*p[1] + p[0]*p[2] + p[1] + p[2] - 1)/(p[3]*(p[1] + p[2])*(p[1] + p[2] - 1))
    ssS = p[0]*(p[1]+p[2])/(1-p[1]-p[2])
    return [0,ssX],[1,ssS]

def trayectory(x0,s0,p,dt=0.02,end_criteria=1.0E-5):
    ## Could use scipy's solve_ivp 
    time,X,S = [0],[x0],[s0]
    dX,dS = stream(X[-1],S[-1],p)
    
    while np.sqrt(dX*dX + dS*dS) > end_criteria:      
        time.append(time[-1]+dt)
        X.append(X[-1]+(dX*dt))
        S.append(S[-1]+(dS*dt))
        dX,dS = stream(X[-1],S[-1],p)
        if len(X) > 10000: break
    
    return pd.DataFrame({'t':time,'x(t)':X,'s(t)':S})

In [19]:
xlin = np.linspace(-0.1,1.1,61)
slin = np.linspace(-0.1,1.1,61)
x,s  = np.meshgrid(xlin,slin)

In [6]:
df = pd.DataFrame()
df["Index"]  = ['Y','q','Ks','b','Q/V','S0']
df.set_index("Index",inplace=True)
df["Label"]  = [r'Y',r'\hat{q}',r'K_s',r'b',r'Q/V',r'S^0']
df["Value"]  = [0.42,10,20,0.15,1000/2000,50]
df["Units"]  = ["mg(X)/mg(S)","mg(S)/mg(X)·d","mg(S)/L","1/d","1/d","mg(S)/L"]
df["Slider"] = [wd.FloatText(value=v,description=f'${k}$') for k,v in zip(df["Label"],df["Value"])]

nd = pd.DataFrame()
nd["Index"]  = np.arange(0,4,1)
nd.set_index("Index",inplace=True)
nd["Label"]  = [fr"\pi_{{{s}}}" for s in nd.index]
nd["Value"]  = np.arange(10,14,1)
nd["Slider"] = [wd.FloatText(value="{:.4f}".format(v),description=f'${k}$') for k,v in zip(nd["Label"],nd["Value"])]

In [9]:
def update_values():
    df["Value"] = [w.value for w in df["Slider"]]
    nd["Value"] = [w.value for w in nd["Slider"]]

def calculate_ps(change):   
    nd["Slider"][0].value = df["Slider"]['Ks'].value/df["Slider"]['S0'].value
    nd["Slider"][1].value = df["Slider"]['b'].value/(df["Slider"]['q'].value*df["Slider"]['Y'].value)
    nd["Slider"][2].value = df["Slider"]['Q/V'].value/(df["Slider"]['q'].value*df["Slider"]['Y'].value)
    nd["Slider"][3].value = 1.0
    
    update_values()

In [10]:
for slider in df["Slider"]:
    slider.observe(calculate_ps,'value')

calculate_ps(True)

In [11]:
whitespace = wd.HTML("&nbsp;&nbsp;<b>&#8594;</b>&nbsp;&nbsp;")
layout = wd.Layout(justify_content='center',align_items='center')

eqTex0 = wd.HTMLMath(r'''\begin{equation}
\begin{array}{rcl}
    \dfrac{dX}{dt} &=& Y\hat{q}X\dfrac{S}{K_S + S} - bX - \tfrac{Q}{V}X \\
    \dfrac{dS}{dt} &=& \tfrac{Q}{V}(S_\infty-S) - \hat{q}X\dfrac{S}{K_S + S}
\end{array}
\end{equation}''',layout=layout)

eqTex1_1 = wd.HTMLMath(r'''\begin{equation*}
\begin{array}{rcl}
    s &=& S/S_\infty \\
    x &=& X/X_{\rm max} \\
    \tau &=& \hat{q}Yt \\
\end{array}
\end{equation*}''')

eqTex1_2 = wd.HTMLMath(r'''\begin{equation*}
\begin{array}{rcl}
    \pi_0 &=& K_S/S_\infty \\
    \pi_1 &=& b/\hat{q}Y \\
    \pi_2 &=& Q/V\hat{q}Y \\
    \pi_3 &=& X_{\rm max}/S_\infty Y \\
\end{array}
\end{equation*}''')

eqTex1 = wd.HBox([eqTex1_1,whitespace,eqTex1_2],layout=layout)

eqTex2 = wd.HTMLMath(r'''\begin{equation}
\begin{array}{rcl}
    \dfrac{dx}{d\tau} &=& x\dfrac{s}{\pi_0 + s} - \pi_1 x - \pi_2 x \\
    \dfrac{ds}{d\tau} &=& \pi_2 \left(1-s\right) - \pi_3 x\dfrac{s}{\pi_0 + s}\\
\end{array}
\end{equation}''',layout=layout)

eqTex3 = wd.HTMLMath(r'''\begin{equation}
\begin{array}{cc}
    x=0 & s=1 \\
    x = \dfrac{\pi_{2} \left(\pi_{0} \pi_{1} + \pi_{0} \pi_{2} + \pi_{1} + \pi_{2} - 1\right)}{\pi_{3} \left(\pi_{1} + \pi_{2}\right) \left(\pi_{1} + \pi_{2} - 1 \right)}& s = \dfrac{\pi_{0} \left(\pi_{1} + \pi_{2}\right)}{1 - \pi_{1} - \pi_{2}}
\end{array}
\end{equation}''',layout=layout)

eqTexs = [eqTex0,eqTex1,eqTex2,eqTex3]

In [27]:
kw_streams = {'density':1,'linewidth':1.5,'arrowsize':0.8,
              'arrowstyle':'->','color':[1,1,1,0.5],'minlength':0.2}

output = wd.Output(layout=layout)

@output.capture(clear_output=True)
def plotStream(x,s):
    update_values()
    
    p = nd["Value"]
    dx,ds = stream(x,s,p)
    ssx,sss = steadyState(p)
    tray = trayectory(1.0E-4,1,p)
    
    kw_streams = {'density':2,'linewidth':1.5,'arrowsize':0.8,
              'arrowstyle':'->','color':'k','minlength':0.1}

    fig = plt.figure(figsize=[12,6])
    gs = gridspec.GridSpec(6, 12)
    ax = fig.add_subplot(gs[:,:6])
    ax.set_aspect("equal")
    ax.streamplot(x,s,dx,ds,**kw_streams,zorder=2)
    ax.scatter(ssx,sss,s=250,clip_on=False,ec='k')
    
    if ssx[1] < 0:
        ax.annotate("Washout",
                  xy=(0.0, 0.99), xycoords='data',
                  xytext=(0.5, 0.5), textcoords='data',
                  size=20, va="center", ha="center",
                  bbox=dict(boxstyle="round4", fc="pink", ec="red"),
                  arrowprops=dict(arrowstyle="-|>",
                                  connectionstyle="arc3,rad=-0.2",
                                  fc="w",ec='red'))
    else:
        ax.plot(tray["x(t)"],tray["s(t)"],lw=5,c='purple')

    ax.grid(True,ls='dashed',lw=1,c=[0.5,0.5,0.5,0.5])
    ax.set_xlabel("$x$ [-]",)
    ax.set_ylabel("$s$ [-]")
    ax.set(xlim=[0,1],ylim=[0,1])   

    pax = fig.add_subplot(gs[:,7:])
    pax.set_facecolor("#F5DEB343")
    pax.plot(tray['t'],tray['x(t)'],label=r"$x(t)$")
    pax.plot(tray['t'],tray['s(t)'],label=r"$s(t)$")
    pax.set_ylim([0,1])
    pax.set_xlim(left=0)
    pax.set_xlabel(r'$t$')#,labelpad=-8)
    pax.legend(loc='upper right',fontsize=12)
    
    plt.show();
  
    return None

def updateFig(b):
    plotStream(x,s);

updateFig(True);

plotButton = wd.Button(description="Plot!",icon='drafting-compass')
plotButton.on_click(updateFig)

gs = wd.GridspecLayout(12,2)
gs[0:5,0] = wd.VBox(list(df["Slider"]))
gs[0:4,1] = wd.VBox(list(nd["Slider"]))
gs[4,1] = plotButton
gs[5:,:] = wd.HBox([output])

accordion = wd.Accordion(eqTexs,layout=wd.Layout(align_items='stretch'))
accordion.selected_index = None
accordion.set_title(0,'Governing ODEs')
accordion.set_title(1,'Nondimensionalization')
accordion.set_title(2,'Non-dimensional ODEs')
accordion.set_title(3,'Steady-state solution')

mainBox = wd.VBox([accordion,gs])
display(mainBox)